In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
from lxml import etree





In [68]:
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")

In [69]:
#open page
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://digital.oegbverlag.at/login?r=https%3A%2F%2Fdigital.oegbverlag.at%2Fkvsystem")
time.sleep(3)

#for some reason I have to manually click on the "reject cookies" option, can't find a way to make the webdriver do it 
#select_element = driver.find_element(By.XPATH, '//*[@id="uc-center-container"]/div[2]/div/div/div/div/button[2]')
#select_element.click()


In [70]:
#select "für ögbverlag kunden"
element=WebDriverWait(driver, 10).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button')
select_element.click()

In [75]:
#login
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")

username.send_keys("jakob.wall@econ.uzh.ch")
password.send_keys("D00rcityoverhere")

driver.find_element(By.ID, "login").click()

In [7]:
#select "erweiterte Suche and search"
element=WebDriverWait(driver, 10).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]')
select_element.click()


In [ ]:
all_years = []
for year in range(1995,2022):
    driver.get("https://digital.oegbverlag.at/kvsystem?q=&v="+str(year)+"-01-01")
    #print("01.01." + str(year))

    #select date
    #date = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/div[2]/div/input")
    #date.send_keys(Keys.CONTROL + "a")
    #date.send_keys(Keys.DELETE)
    #date.send_keys("01.01." + str(year))

    #search
    element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]/div/input")))
    searchbar = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/input")
    searchbar.send_keys(Keys.RETURN)

    #I need to click on "mehr anzeigen" until the whole list is visible. 
    #Unfortunately, the site does not work perfectly, and keeps on displaying the button "mehr anzeigen" as a clickable element even though we are at the end. As a solution, I just clik 40 times, which is always enough
    #  Unfortunately, the site does not work perf
    #select "mehr anzeigen"
    for i in range (40):
        element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button")))
        select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button')
        select_element.click()
        #need to wait a bit, otherwise the site doesn't respond
        time.sleep(2)
        print(i)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    links = soup.find_all('a')
    # Create a list to store (text, link) tuples
    result_list = []
    # Loop through each <a> tag
    for link in links:
        # Get the href attribute (the link)
        href = link.get('href')
        
        # Get the text inside the <span> (if any)
        text = link.get_text().strip()  # .strip() removes any extra whitespace
        
        if href and text:
            result_list.append((text, href))

    # Filter the list to keep only elements where the link starts with "/kvsystem"
    filtered_list = [(text, link) for text, link in result_list if link.startswith("/kvsystem/")]

    #append current year
    all_years.append(filtered_list)

    


In [ ]:
#turn list into pandas dataframe 
all_years_df = pd.DataFrame(columns = ['Text', 'Link', 'Year'])
for i in range(len(all_years)):
    df_current = pd.DataFrame(all_years[i], columns=['Text', 'Link'])
    df_current['Year'] = 1995+i

    all_years_df = pd.concat([all_years_df,df_current ], ignore_index=True)

In [26]:
#save as csv
all_years_df.to_csv('./data/kv_links_all_years.csv')

In [27]:

#load csv and sort
all_years_df = pd.read_csv('./data/kv_links_all_years.csv')
all_years_df = all_years_df.sort_values('Text')
all_years_df = all_years_df.reset_index(drop=True)
all_years_df = all_years_df.drop(columns =['Unnamed: 0'])

#ignore zusatz kvs
all_years_df = all_years_df[~all_years_df['Text'].str.contains("ZKV")]
all_years_df = all_years_df.reset_index(drop=True)


In [3]:
contracts_df = pd.DataFrame()

In [ ]:
for i in range(1,len(all_years_df)):
    print(i)
    link = str("https://digital.oegbverlag.at" + all_years_df['Link'].iloc[i])

    driver.get(link)
    time.sleep(1.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #check if we get a server error: if yes, we skip to the next link
    is_server_error = 1 if "Serverfehler" in soup.title.string else 0
    if is_server_error:
        print("Server Error")

    else:
        #'_c._d._e._f._h._av._al'
        #'_go._p._c._d._e._ap._gn._i._bs'
        element=WebDriverWait(driver, 300).until(EC. element_to_be_clickable((By.CLASS_NAME, '_c._d._e._f._h._av._al')))
        xpath = "//div[contains(concat(' ', normalize-space(@class), ' '), ' _c ') and contains(concat(' ', normalize-space(@class), ' '), ' _d ') and contains(concat(' ', normalize-space(@class), ' '), ' _e ')]/div[contains(concat(' ', normalize-space(@class), ' '), ' _c ') and contains(concat(' ', normalize-space(@class), ' '), ' _d ') and contains(concat(' ', normalize-space(@class), ' '), ' _e ') and contains(concat(' ', normalize-space(@class), ' '), ' _f ') and contains(concat(' ', normalize-space(@class), ' '), ' _h ') and contains(concat(' ', normalize-space(@class), ' '), ' _av ') and contains(concat(' ', normalize-space(@class), ' '), ' _al ')]"
        subcontracts = driver.find_elements(By.XPATH, xpath)
        #subcontracts = driver.find_elements(By.CLASS_NAME, '_c._d._e._f._h._av._al')

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        css_selector = f".{'_c._d._e'} > .{'_c._d._e._f._h._av._al'}"
        subcontracts_soup = soup.select(css_selector)

        # Step 3: Loop over the subparts
        contract_df = pd.DataFrame(columns= ['Text', 'Link', 'Year', 'subcontract_html', 'subcontract_text'])
        subcontracts_loop = subcontracts
        subcontract_df = pd.DataFrame(columns=['contract', 'year', 'subcontract_title', 'subcontract_html', 'subcontract_text'])
        for subcontract_nr in range(len(subcontracts)):
            #print(subcontract_nr)
            #load page
            driver.get(link)
            element=WebDriverWait(driver, 300).until(EC. element_to_be_clickable((By.CLASS_NAME, '_c._d._e._f._h._av._al')))
            subcontracts_loop = driver.find_elements(By.XPATH, xpath)

            subcontract = subcontracts_loop[subcontract_nr]
            # Step 3a: Click on the subpart to load its content
            subcontract.click()

            # Wait for the page to load 
            element=WebDriverWait(driver, 300).until(EC. element_to_be_clickable((By.CLASS_NAME, '_c._d._e._f._h._av._al')))

            #Step 3b: Scrape the displayed text on the subpart
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            #paragraphs = soup.find_all(class_=re.compile(r'\bcp_digital_para para pr-'))

            #print(paragraphs)

            #select text of the actual subcontract
            actual_subcontract = soup.find(class_=re.compile(r'\bcp_digital_para cp_digital'))

            subcontract_df.loc[subcontract_nr,'contract'] = all_years_df.loc[i,'Text']
            subcontract_df.loc[subcontract_nr,'year'] = all_years_df.loc[i,'Year'] 
            subcontract_df.loc[subcontract_nr,'subcontract_title'] = subcontracts_soup[subcontract_nr].find('a').get_text()
            subcontract_df.loc[subcontract_nr,'subcontract_html'] = str(actual_subcontract)
            subcontract_df.loc[subcontract_nr, 'subcontract_text'] = actual_subcontract.get_text()


        contracts_df = pd.concat([contracts_df, subcontract_df], ignore_index = True)



In [8]:
#subset to main contract ("Rahmen") only
contracts_df = contracts_df[['contract', 'year', 'subcontract_title', 'subcontract_html', 'subcontract_text']]

rahmen_all_df = contracts_df[contracts_df['subcontract_title'] == 'Rahmen']

In [3]:
#save
#contracts_df.to_csv('./data/kv_contracts_all_years.csv', index=False)
#rahmen_all_df.to_csv('./data/kv_contracts_rahmen_all_years.csv', index=False)
contracts_df = pd.read_csv('./data/kv_contracts_all_years.csv')